In [0]:
df = spark.read.table("mk_fiddles.detroit_911.incidents_bronze")
display(df)

Databricks visualization. Run in Databricks to view.


## Silver Steps
1. Filter out bad districts and priorities
2. Create a date and a time from the timestamp

## Gold Steps
1. Filter out anything that matches today
2. Aggregate by district, priority and hour
3. Interpolate date timestamps hourly

In [0]:
from pyspark.sql.functions import col, concat, expr, lit

df_silver = (df
 .filter(col("priority").between("1","5"))
 .filter(col("council_district").between(1, 7))
 .withColumn("council_district", concat(lit("D"), col("council_district").cast("int").cast("string")))
 .withColumn("priority", concat(lit("P"), col("priority")))
 .withColumn("called_at_timestamp", (col("called_at") / 1000).cast("timestamp"))
 .withColumn("called_at_date", col("called_at_timestamp").cast("date"))
 .withColumn("called_at_hour", expr("date_trunc('HOUR', called_at_timestamp)"))
 )

display(df_silver)

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
df_aggregate = (df_silver
 .groupBy("council_district", "priority", "called_at_hour")
 .count()
 .withColumnRenamed("count", "call_count")
)

In [0]:
import datetime as dt
from pyspark.sql.functions import sequence, explode, col, lit

min_date, max_date = df_silver.selectExpr( "min(called_at_hour)", "max(called_at_hour)").first()

latest_allowed_date = (dt.datetime.now() - dt.timedelta(days=1)).date()

min_date = min_date.strftime("%Y-%m-%d %H:%M:%S")
max_date = max_date.strftime("%Y-%m-%d %H:%M:%S")
districts = df_silver.select("council_district").distinct()
priorities = df_silver.select("priority").distinct()

date_range = (
  spark.range(1)
      .select(
          explode(
          sequence(
              lit(min_date).cast("timestamp"),
              lit(max_date).cast("timestamp"),
              expr("interval 1 hour")
          )
      ).alias("called_at_hour")
))

interpolate_df = (
  date_range
  .crossJoin(districts)
  .crossJoin(priorities)
)

df_gold = (
  interpolate_df
  .join(df_aggregate, on=["council_district", "priority", "called_at_hour"], how="left")
  .fillna(0)
  .filter(col("called_at_hour").cast("date") <= latest_allowed_date)
  .orderBy("council_district", "priority", "called_at_hour")
)

display(df_gold)



Databricks visualization. Run in Databricks to view.